In [1]:
#steps

# Import large Mt5 model
# bring in dataset
# Fine tune
# Evaluate

## Data Conversion

In [2]:
##Bring in dataset

from datasets import load_dataset

from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

dataset = load_dataset("squad_es",'v1.1.0')

# Step 1 and 2: Combine context and question, format the answer
from transformers import T5TokenizerFast

# Load the tokenizer
tokenizer = T5TokenizerFast.from_pretrained('google/mt5-base')

Reusing dataset squad_es (/root/.cache/huggingface/datasets/squad_es/v1.1.0/1.1.0/bcada4f600192451443b95e24f609325705c5185b8aad97bffa8bc3784a867ad)


  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
dataset['train']['answers'][0]['text'][0]

'Santa Bernadette Soubirous'

In [4]:

# Step 1 and 2: Combine context and question, format the answer
from transformers import T5TokenizerFast

# Load the tokenizer
tokenizer = T5TokenizerFast.from_pretrained('google/mt5-base')


def prep_data(train_subset):

    # Create new lists for our inputs and answers
    input_ids = []
    input_attn_msk = []
    answers_ids = []

    for id_value, title_value, context_value, q_value, a_value in zip(train_subset['id'], train_subset['title'], train_subset['context'],train_subset['question'],train_subset['answers']):

        #INPUTS
        # Combine the context and question into a single string
        input_str = context_value + " " + q_value
        #print(input_str)

        #tokenize the inputs
        inputs = tokenizer(input_str, truncation=True, padding='max_length', max_length=512)
        #print(inputs.input_ids)
        input_ids.append(inputs.input_ids)
        input_attn_msk.append(inputs.attention_mask)


        #ANSWERS
        answers = a_value['text'][0]
        answers_tokens = tokenizer(answers, truncation=True, padding='max_length', max_length=128)
        answers_ids.append(answers_tokens.input_ids)


    result_list = []


    # Iterate over the values from each list
    for input_id, attention_mask, label  in zip(input_ids, input_attn_msk, answers_ids):
        # Create a dictionary with the desired keys
        data_dict = {
            'input_ids': torch.tensor(input_id),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(label)

        }

        # Append the dictionary to the result list
        result_list.append(data_dict)

    return(result_list)


    


In [5]:
import torch
# Start with a subset of your data for testing
train_subset = dataset['train'][:100]  # Adjust this as needed

model_train_data = prep_data(train_subset)


# Start with a subset of your data for testing
val_subset = dataset['validation'][:10]  # Adjust this as needed

model_val_data = prep_data(val_subset)
          
          

## Data ready test

In [6]:
# replace 'train_dataset' with your dataset variable
sample = model_train_data[0]

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("Your dataset appears to be correctly structured for Trainer.")


Your dataset appears to be correctly structured for Trainer.


In [7]:
# replace 'train_dataset' with your dataset variable
sample = model_val_data[0]

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("Your dataset appears to be correctly structured for Trainer.")


Your dataset appears to be correctly structured for Trainer.


In [8]:
print(model_val_data[0]['input_ids'].shape)
print(model_val_data[0]['attention_mask'].shape)
print(model_val_data[0]['labels'].shape)
                        

torch.Size([512])
torch.Size([512])
torch.Size([128])


In [9]:
model_train_data

[{'input_ids': tensor([   259, 220933,   1790,  32294,    261,    283,    259,  60876,   2812,
             335,    259,  58878,  41742,  10292, 108405,    260,    642,  22472,
             269,    283,  37795,  54126,    269,  16347,    426,    259,  39909,
             268,   6017,   2318,    573,  99834,    262, 123281,    262,    269,
             283,  89306,    278,    259,  24952,    260,    563,  87060,    259,
           63047,    426,    259,  39909,    268,   6017,    259,    276,    259,
           20189,    259,    262,   7938,    261,   2318,    573,  99834,    262,
             269,  56333,    265,    269,  39333,    450,    595,    259,  98838,
             263,  57692,   2131,    450,    283,    340, 116073,    313,  30702,
           15782,   4515,   1517,   3548,   1865,   1191,   4143,    476,    440,
             259,  39909,    268,   6017,   1957,    283,    364,   7590,  33858,
             426,   1560,  15905,   7090,  56019,    260,    563,  87060,    269,
   

## Fine Tuning

In [10]:
from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the model
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

# Initialize the training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=model_train_data,            # training dataset
    eval_dataset=model_val_data,         # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# Train the model
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 75
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jprivera44 (cs7643_jp). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss
10,48.656100
20,47.316400
30,47.761800
40,47.493800
50,46.993100
60,46.464100
70,46.140400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=75, training_loss=47.157890625, metrics={'train_runtime': 26.011, 'train_samples_per_second': 11.534, 'train_steps_per_second': 2.883, 'total_flos': 359713839513600.0, 'train_loss': 47.157890625, 'epoch': 3.0})

In [11]:
#saving data to disk

import torch

# Save the model and training_args
#torch.save(model.state_dict(), 'model_input/model.pth')
model.save_pretrained('model_input/model')
torch.save(training_args, 'model_input/training_args.pth')

import pickle

with open('model_input/train_data.pkl', 'wb') as f:
    pickle.dump(model_train_data, f)

with open('model_input/val_data.pkl', 'wb') as f:
    pickle.dump(model_val_data, f)

# Here, compute_metrics is a function. You cannot save it directly, but you can define it in your script or import it from a shared Python module.


Configuration saved in model_input/model/config.json
Model weights saved in model_input/model/pytorch_model.bin


In [12]:
stop

NameError: name 'stop' is not defined

In [14]:
#call the file with nsys to track performance

!nsys profile --trace=cuda --stats=true python3 Mt5-fine-tuning.py --model 'model_input/model' --training_args 'model_input/training_args.pth' --train_data_path 'model_input/train_data.pkl' --val_data_path 'model_input/val_data.pkl'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of the model checkpoint at model_input/model were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementatio

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=model_train_data,            # training dataset
    eval_dataset=model_val_data,         # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
